In [1]:
from itertools import chain, combinations
from collections import defaultdict
from heapq import nlargest
from operator import itemgetter
import math
import pandas as pd
import ml_metrics as metrics

In [2]:
train = pd.read_csv("50k_train.csv")
test = pd.read_csv("50k_test.csv")
topclusters = list(train.hotel_cluster.value_counts().head().index)

In [3]:
ulc_odd = defaultdict(lambda: defaultdict(int))
all_columns = defaultdict(lambda: defaultdict(int))
sdi_hc_hm_by = defaultdict(lambda: defaultdict(int))
sdi_dict = defaultdict(lambda: defaultdict(int))
hc_dict = defaultdict(lambda: defaultdict(int))
popular_hotel_cluster = defaultdict(int)
all_minus_odd = defaultdict(lambda: defaultdict(int))
all_minus_hm = defaultdict(lambda: defaultdict(int))
all_minus_ulci = defaultdict(lambda: defaultdict(int))
all_minus_ulco = defaultdict(lambda: defaultdict(int))
all_minus_ulr = defaultdict(lambda: defaultdict(int))
for index, row in train.iterrows():
    values = []
    hotel_cluster = row['hotel_cluster']
    ulci = row['user_location_city']
    ulco = row['user_location_country']
    ulr = row['user_location_region']
    odd = row['orig_destination_distance']
    sdi = row['srch_destination_id']
    hm = row['hotel_market']
    hc = row['hotel_country']
    by = row['year']

    if ulci != '' and odd != '':
        ulc_odd[(ulci, odd)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and odd != '' and hm != '':
        all_columns[(ulci, ulco, ulr, odd, hm)][hotel_cluster] += 1
        
    if sdi != '' and hc != '' and hm != '' and by != '':
        sdi_hc_hm_by[(sdi, hc, hm, by)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_odd[(ulci, ulco, ulr, sdi, hm)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and odd != '':
        all_minus_hm[(ulci, ulco, ulr, sdi, odd)][hotel_cluster] += 1
    
    if hm != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_ulci[(hm, ulco, ulr, sdi, odd)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_ulco[(hm, ulci, ulr, sdi, odd)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_ulr[(hm, ulci, ulco, sdi, odd)][hotel_cluster] += 1
        
    if sdi != '':
        sdi_dict[sdi][hotel_cluster] += 1

    if hc != '':
        hc_dict[hc][hotel_cluster] += 1

In [4]:
preds = []
indicies = []
for index, row in test.iterrows():
    output = []
    filled = []
    run_ml = True
    hotel_cluster = row['hotel_cluster']
    ulci = row['user_location_city']
    ulco = row['user_location_country']
    ulr = row['user_location_region']
    odd = row['orig_destination_distance']
    sdi = row['srch_destination_id']
    hm = row['hotel_market']
    hc = row['hotel_country']
    
    if (ulci, odd) in ulc_odd:
        run_ml = False
        individual_outputs = []
        clusters = ulc_odd[ulci, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    '''       
    if (ulci, ulco, ulr, odd, hm) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_columns[ulci, ulco, ulr, odd, hm]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    
    if (ulci, ulco, ulr, sdi, hm) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_odd[ulci, ulco, ulr, sdi, hm]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (ulci, ulco, ulr, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_hm[ulci, ulco, ulr, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (hm, ulco, ulr, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_ulci[hm, ulco, ulr, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (hm, ulci, ulr, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_ulco[hm, ulci, ulr, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (hm, ulci, ulco, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_ulr[hm, ulci, ulco, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    '''

    if (sdi, hc, hm, by) in sdi_hc_hm_by:
        run_ml = False
        individual_outputs = []
        clusters = sdi_hc_hm_by[sdi, hc, hm, by]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    '''
    if sdi in sdi_dict:
        run_ml = False
        individual_outputs = []
        clusters = sdi_dict[sdi]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if hc in hc_dict:
        run_ml = False
        individual_outputs = []
        clusters = hc_dict[hc]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    '''
    
    for i in range(len(topclusters)):
        if topclusters[i] in filled:
            continue
        if len(filled) == 5:
            break
        output.append(str(topclusters[i]))
        filled.append(topclusters[i])
    if run_ml:
        indicies.append(index)
    preds.append(output)

In [5]:
preds_file = []
for i in range(len(preds)):
    row_pred = []
    row_pred.append(int(preds[i][0]))
    row_pred.append(int(preds[i][1]))
    row_pred.append(int(preds[i][2]))
    row_pred.append(int(preds[i][3]))
    row_pred.append(int(preds[i][4]))
    preds_file.append(row_pred)

In [6]:
target = [[l] for l in test["hotel_cluster"]]
print(metrics.mapk(target, preds_file, k=5))

0.249236666667


In [7]:
len(indicies)

871